### https://www.kaggle.com/c/tmdb-box-office-prediction

https://www.kaggle.com/c/tmdb-box-office-prediction/overview/evaluation - explain about RMSLE

In [1]:
import pandas as pd 
from sklearn import datasets
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection  import cross_val_score
from sklearn.metrics import mean_squared_log_error


from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor 
#def rmsle_score(y, y0):
#    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 5400)
np.set_printoptions(threshold=np.inf)

In [2]:
#Import datasets
df_train = pd.read_csv('TMDB_Box_Office/train.csv')
df_test = pd.read_csv('TMDB_Box_Office/test.csv')

In [3]:
df_train.head(2)

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435


In [4]:
df_train.shape, df_test.shape

((3000, 23), (4398, 22))

In [5]:
df_y_train = df_train[['id','revenue']].set_index('id')

df_train  = df_train.set_index('id')
df_test  = df_test.set_index('id')

In [6]:

cols  = ['budget', 'original_language' ,'popularity', 'runtime', 'status']
df_train  = df_train[cols]
df_test  = df_test[cols]
#display(train.iloc[5])

#One-Hot-Encoding for all nominal data
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)
display(df_train.shape,df_test.shape)


df_train.fillna(0)
_= df_test.fillna(0)


(3000, 41)

(4398, 45)

,budget,popularity,runtime,original_language_ar,original_language_bn,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_it,original_language_ja,original_language_ko,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_tr,original_language_ur,original_language_vi,original_language_zh,status_Released,status_Rumored
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,14000000,6.575393,93.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,40000000,8.248895,113.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,3300000,64.299990,105.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1200000,3.174936,122.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,0,1.148070,118.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,8000000,0.743274,83.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,14000000,7.286477,92.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,0,1.949044,84.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,0,6.902423,100.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [7]:
#Create columns in train/test dataframes if they only exist in one of them (can happen through one hot encoding / get_dummies)
#  Example: There are no status=`Post Production` entries in the training set, but there are some in the test set.
df_train, df_test = df_train.align(df_test, join='outer', axis=1, fill_value=0)
display(df_train.shape,df_test.shape)

# 4.) Show the first rows of one of the prepared tables
df_train.head(2)



(3000, 50)

(4398, 50)

,budget,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tr,original_language_ur,original_language_vi,original_language_xx,original_language_zh,popularity,runtime,status_Post Production,status_Released,status_Rumored
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,14000000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.575393,93.0,0,1,0
2,40000000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.248895,113.0,0,1,0


In [8]:

# 1.) Remove table meta data, column names etc. → Just use values for prediction.
train_x_values = df_train.values
train_y_values = df_y_train.values

test_x_values  = df_test.values



In [9]:

# Create Validation Split
X_train, X_val, y_train, y_val = train_test_split(train_x_values, train_y_values, test_size=0.5, random_state=56)
#display('before scale',X_train[0])

# Scale all X values
X_scaler = StandardScaler()
X_train_scaled  = X_scaler.fit_transform(X_train)
X_val_scaled    = X_scaler.transform(X_val)
X_test_scaled   = X_scaler.transform(test_x_values)
#display('after scale',X_train_scaled[0])



In [10]:
#display('before scale',X_train[0])
#display('after scale',X_train_scaled[0])

In [11]:
# transform and convert column-vector y to a 1d array with ravel
y_scaler = MinMaxScaler((0,1)) 
y_train_scaled  = y_scaler.fit_transform(np.log(y_train)).ravel() 
#y_val_scaled  = y_scaler.transform(np.log(y_val)).ravel() #not used but here for consistency



In [12]:
np.any(np.isnan(X_train_scaled))
np.isnan(X_train_scaled)

True

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, Fal

In [13]:

#np.all(np.isfinite(X_train_scaled))
#np.isfinite(X_train_scaled)
#np.where(np.isfinite(X_train_scaled))
np.isfinite( X_train_scaled ).all()

False

In [14]:
X_train_scaled = np.nan_to_num(X_train_scaled)
X_val_scaled = np.nan_to_num(X_val_scaled)
X_test_scaled = np.nan_to_num(X_test_scaled)

In [15]:
# Calculate the coefficients of the linear regression / "Train"
reg  = KNeighborsRegressor().fit(X_train_scaled, y_train_scaled)

In [16]:
# Define functions to calculate a score - use Root Mean squared logarithmic error (RMSLE) regression loss
def score_function(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

def score_function2(y_true, y_pred):
    # alternative implementation
    y_pred = np.where(y_pred>0, y_pred, 0)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

def inverseY(y):
    #m = log(x)  e^m = X
    #y_train_scaled  = y_scaler.fit_transform(np.log(y_train)).ravel() 
    return np.exp(y_scaler.inverse_transform(np.reshape(y, (-1,1))))






In [17]:
#Apply the regression model on the prepared train, validation and test set and invert the logarithmic scaling
y_train_pred  = inverseY(reg.predict(X_train_scaled))
y_val_pred    = inverseY(reg.predict(X_val_scaled))
y_test_pred   = inverseY(reg.predict(X_test_scaled))
                   
#Print the RMLS error on training, validation and test set. it should be as low as possible
print("RMLS Error on Training Dataset:\t", score_function(y_train , y_train_pred), score_function2(y_train, y_train_pred))
print("RMLS Error on Val Dataset:\t", score_function(y_val , y_val_pred), score_function2(y_val , y_val_pred))
print("RMLS Error on Test Dataset:\t Check by submitting on kaggle")

RMLS Error on Training Dataset:	 2.114647718079455 2.114647718079455
RMLS Error on Val Dataset:	 2.3991537004096277 2.3991537004096277
RMLS Error on Test Dataset:	 Check by submitting on kaggle


In [18]:
df_test = original_df_test_X.assign(revenue=y_test_pred)

# 2.) Extract a table of ids and their revenue predictions
df_test_y = df_test[['id','revenue']].set_index('id')

# 3.) save that table to a csv file. On Kaggle, the file will be visible in the "output" tab if the kernel has been commited at least once.
df_test_y.to_csv("submission.csv")

# 4.) output the head of our file her to check if it looks good :)
pd.read_csv("submission.csv").head(5)

NameError: name 'original_df_test_X' is not defined

In [ ]:


X = df.drop('revenue',axis=1)
y = df.revenue

X_train,X_test,y_train ,y_test = train_test_split(X,y,random_state = 42,train_size = 0.8,test_size=0.2)
X_train,X_test ,y_train ,y_test

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)
scores = cross_val_score(lin_reg, X, y, cv = 5)
    

#scores = cross_val_score(lin_reg,X_train,y_train,cv =10,scoring=rmsle_score)
#scores,scores.mean(),scores.std()
